In [ ]:
import sys
import os
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc
from utils.util import calc_cw_score
from utils.util import scale_versability, scale_compatibility
import json
import glob
import random
import pandas as pd
from utils.util import get_category
from utils.util import calc_compatibility_score
from utils.util import get_progress_percent

In [ ]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import calc_cw_score
from utils.util import scale_versability, scale_compatibility

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [ ]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [ ]:
id_to_vector_compatibility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [ ]:
items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in list(map(is_target_category, json_dict["items"])):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [ ]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [ ]:
# (cw, proposal_score, topic_model_score)の配列
com_score = []

N = 50000
for i in range(N):
    if i % 100 == 0:
        print(get_progress_percent(i, N), end='')
    dataset = {
        'tops':random.sample(items["tops"], 1000),
        'bottoms':random.sample(items["bottoms"], 1000),
        'shoes':random.sample(items["shoes"], 1000),
    }

    # cwの初期化
    cw = {
        'tops':random.sample(dataset["tops"], T),
        'bottoms':random.sample(dataset["bottoms"], T),
        'shoes':random.sample(dataset["shoes"], T),
    }
                    
    c = calc_compatibility_score(cw, id_to_vector_compatibility)
    # print('======')
    # print(c, v)
    # c, v = scale_compatibility(v), scale_versability(v)
    # print(c, v)
    # print('======')
    com_score.append(c)